**Task 1**

So according to the given sensor/measurement model in equation 1

p(z=1|x=1)=0.6, this means that the door is at open state and the probability that the sensor also measure the door to be open is 60%.

p(z=0|x=1)=0.4, this means that the door is at open state and the probability that the sensor measures the door to be close is 40%.

p(z=1|x=0)=0.2, this means that the door is at close state and the probability that the sensor measures the door to be open is 20%.

p(z=0|x=0)=0.8, this means that the door is at close state and the probability that the sensor measures the door to be close is 80%.

These above equations indicate us that the sensor is reliability to detect the closed doors is better with 80% accuracy whereas  when the door is open it detects the state of the door correctly with just 60% accuracy, in this case the reliability for detecting open door is low. Therefore we have better relaibility for close door rather than detecting open door.

**For Part B**

We have two control option that is u = [push, do nothing] , and the states are x = [open, closed].

Now we have the state transition probabilities that is for u = push:

$p(x_{t} = open | x_{t-1} = open) = 1 $

$p(x_{t} = close | x_{t-1} = open) = 0$

$p(x_{t} = open | x_{t-1} = close) = 0.8$

$p(x_{t} = close | x_{t-1} = close) = 0.2$

Now if we write the probability of x and x_prev_state in a matrix form for u = push:

$\begin{pmatrix}
  x_{t1} \\
  x_{t2} \\
 \end{pmatrix} =
 A
 \begin{pmatrix}
  x_{t1-1} \\
  x_{t2-1} \\
 \end{pmatrix}
 $

$\begin{pmatrix}
  x_{t1} \\
  x_{t2} \\
 \end{pmatrix} =
 \begin{pmatrix}
  1 & 0.8 \\
  0 & 0.2 \\
 \end{pmatrix}
 \begin{pmatrix}
  x_{t1-1} \\
  x_{t2-1} \\
 \end{pmatrix}
 $

 $A=\begin{pmatrix}
  1 & 0.8 \\
  0 & 0.2 \\
 \end{pmatrix}$

 where $x_{t1}$ is open state and $x_{t2}$ is close state.

 For case 2 u = do nothing the state remains the same so the A matrix would be Identity matrix.

 $\begin{pmatrix}
  x_{t1} \\
  x_{t2} \\
 \end{pmatrix} =
 \begin{pmatrix}
  1 & 0 \\
  0 & 1 \\
 \end{pmatrix}
 \begin{pmatrix}
  x_{t1-1} \\
  x_{t2-1} \\
 \end{pmatrix}
 $

 **In both the cases we observe that the system is Linear and it follows the markov assumption.**


**Task 2**
1. In question 1 it requires  9 iterations to reach 99.99% certainty for the
   door being open,  when the action is do nothing and the the door is open

2. In question 2 it requires  4 iterations to reach 99.99% certainty for the door being open,  when the action is push and the the door is open
3. In question3 it requires 10 iterations to attain a steady state where it reaches 99.9955% and stating the belief of the door to be **open**.


In [43]:
import numpy as np

def predict(beliefs, action, transition_probs):
  """
  Predict the next state beliefs based on the current action.

  Parameters:
  - beliefs (list): Current beliefs about the state [open, closed].
  - action (int): Index representing the action taken.
  - transition_probs (list): Transition probabilities matrix.

  Returns:
  - predicted_beliefs (list): Predicted beliefs about the next state [open, closed].
  """


  # Here I make prediction of the door to be in open state
  # For this I take the transition probabilities for the next state for the door being open given the action to predict the belief of door being open
  predicted_open = transition_probs[action][0] * beliefs[0] + transition_probs[action][1] * beliefs[1]


  # Calculate the predicted probability of the door being closed,
  # considering that the sum of probabilities for open and closed states is always 1
  predicted_closed = 1 - predicted_open

  return [predicted_open, predicted_closed]



In [44]:
def update(beliefs, measurement, sensor_probs):
  """
  Update the beliefs based on the sensor measurement.

  Parameters:
  - beliefs (list): Current beliefs about the state [open, closed].
  - measurement (int): Index representing the sensor measurement.
  - sensor_probs (list): Sensor probability matrix.

  Returns:
  - updated_beliefs (list): Updated beliefs about the state [open, closed].
  """

  # Calculate normalization factor for Bayes update rule
  normalization_factor = 1 / (sensor_probs[measurement][0] * beliefs[0] + sensor_probs[measurement][1] * beliefs[1])

  # Update the predicted belief with sensor measurements for open and closed states of the door
  updated_open = normalization_factor * sensor_probs[measurement][0] * beliefs[0]
  updated_closed = 1 - updated_open

  return [updated_open, updated_closed]

# Define the transition probabilities for the next state being open given the current action

transition_probs = [
    [1, 0],            # [p(x = open | u = do_nothing, x_prev_state = open) = 1, p(x = open | u = do_nothing, x_prev_state = closed) = 0]
    [1, 0.8]           # [p(x = open | u = push, x_prev_state = open) = 1, p(x = open | u = push, x_prev_state = closed) = 0.8]
]

# Define sensor probabilities
# [[p(z = closed | x = open) = 0.4, p(z = closed | x = closed) = 0.8],
#  [p(z = open | x = open) = 0.6, p(z = open | x = closed) = 0.2]]

sensor_probabilities = [
    [0.4, 0.8],
    [0.6, 0.2]
]


In [45]:
def bayes_filter(action_measurement_pair,task3=False):

  # Define the initial beliefs
  initial_beliefs = [0.5, 0.5]

  # To keep count for number of iterations
  iteration = 0

  while True:
    # Store the current beliefs for comparison in the next iteration
    prev_updated_beliefs = initial_beliefs

    # Predict step
    predicted_beliefs = predict(initial_beliefs, action_measurement_pair[0], transition_probs)

    # Update step
    updated_beliefs = update(predicted_beliefs, action_measurement_pair[1], sensor_probabilities)

    # Update the initial beliefs for the next iteration
    initial_beliefs = updated_beliefs
    # Increment iteration
    iteration += 1

    # Condition for the first and second question
    if (initial_beliefs[0] >= 0.9999 or initial_beliefs[1] >= 0.9999) and not task3:  # 0.9999 is the threshold for certainty in the belief
      break

    # Condition for the third question where the steady state is to be measured
    if iteration > 1 and abs(prev_updated_beliefs[0] - updated_beliefs[0]) < 1e-4:
      break


  return updated_beliefs, iteration

In [46]:
#Test Cases
#Task 2 First question
# Define possible action and measurement combinations
action_measurement_pair = [0, 1]
belief,iteration=bayes_filter(action_measurement_pair,task3=False) #applying bayes filter
print("Number of iterations for which the robot is certain that the door is open",iteration)

#Task 2 Second question
# Define possible action and measurement combinations
action_measurement_pair = np.array([1, 1])
belief,iteration=bayes_filter(action_measurement_pair,task3=False) #applying bayes filter
print("Number of iterations for which the robot is certain that the door is open",iteration)

#Task 2 Third question
# Define possible action and measurement combinations
action_measurement_pair = np.array([1, 0])
belief,iteration=bayes_filter(action_measurement_pair,task3=True) #applying bayes filter
print("Number of iterations :",iteration, "The steady state certainity :",belief[0])


Number of iterations for which the robot is certain that the door is open 9
Number of iterations for which the robot is certain that the door is open 4
Number of iterations : 10 The steady state certainity : 0.999955060355391
